In [11]:
from flask import Flask, request, jsonify
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import SGDClassifier
import joblib

In [12]:
# Initialize Flask app
app = Flask(__name__)

In [13]:
# Database connection parameters
host = 'localhost'
dbname = 'StarsGalaxiesQuasars'
user = 'postgres'
password = '121792'
port = '5432'

In [14]:
# Create SQLAlchemy engine
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')

In [15]:
# Load and preprocess data
def load_data():
    query = """
    SELECT o.*, c.class
    FROM observation o
    JOIN classification c ON o.classid = c.classid;
    """
    df = pd.read_sql(query, engine)
    df.drop(columns=['observationid', 'classid', 'objid', 'specobjid', 'run', 'rerun', 'camcol', 'field', 'fiberid', 'plate', 'mjd'], inplace=True)
    return df

In [17]:
# Load trained model and scaler
model = joblib.load("svm_model.pkl")
scaler = joblib.load("scaler.pkl")
label_encoder = joblib.load("label_encoder.pkl")

FileNotFoundError: [Errno 2] No such file or directory: 'svm_model.pkl'

In [ ]:
@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json
        df = pd.DataFrame([data])
        df_scaled = scaler.transform(df)
        prediction = model.predict(df_scaled)
        predicted_class = label_encoder.inverse_transform(prediction)[0]
        return jsonify({'prediction': predicted_class})
    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run(debug=True)